#Fine-tuning ResNet50

In [35]:
from torchvision.models import resnet50, ResNet50_Weights

In [36]:
from google.colab import drive 
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [46]:
!unzip -q "/content/gdrive/My Drive/archive.zip"

replace BIRDS-450-(200 X 200)-99.28.h5? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace birds.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [47]:
import json
import torch
from torch import nn
from torch import optim
from torch.utils.data import DataLoader, Dataset

import torchvision
from torchvision import datasets, transforms,models
from torchvision.transforms import ToTensor
from torchvision.utils import make_grid

import tqdm
import itertools
import collections
from IPython import display

import matplotlib.pyplot as plt
import os
import sys
import pandas as pd
import cv2
import time
import numpy as np

In [48]:
dataset_path = '/content'

In [49]:
class_dict = pd.read_csv(dataset_path + "/birds.csv")

In [50]:
class_dict.head()

,class id,filepaths,labels,scientific label,data set
0,0,train/ABBOTTS BABBLER/001.jpg,ABBOTTS BABBLER,Malacocincla abbotti,train
1,0,train/ABBOTTS BABBLER/002.jpg,ABBOTTS BABBLER,Malacocincla abbotti,train
2,0,train/ABBOTTS BABBLER/003.jpg,ABBOTTS BABBLER,Malacocincla abbotti,train
3,0,train/ABBOTTS BABBLER/004.jpg,ABBOTTS BABBLER,Malacocincla abbotti,train
4,0,train/ABBOTTS BABBLER/005.jpg,ABBOTTS BABBLER,Malacocincla abbotti,train


## Dataset preparing

In [148]:
def get_transform():
    return transforms.Compose([transforms.ToTensor()])

# dataset_type must be in ['train', 'valid', 'test']
class Birds450Dataset(Dataset):
    
    def __init__(self, dataset_path , dataset_type = 'train', transforms = None):
        
        super().__init__()

        class_dict = pd.read_csv(dataset_path + "/birds.csv")
        self.path_list = class_dict[class_dict['data set'] == dataset_type]['filepaths'].tolist()
        self.classes = class_dict[class_dict['data set'] == dataset_type]['class id'].tolist()
        self.labels = class_dict[class_dict['data set'] == dataset_type]['labels'].tolist()
        self.transforms = transforms
        
        
    def __getitem__(self, index):
    
        image_path = self.path_list[index]
        
        #Reading image
        image = cv2.imread(image_path, cv2.IMREAD_COLOR)
        image = cv2.resize(image,(224,224), interpolation=cv2.INTER_CUBIC)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB).astype(np.float32)
        image /= 255.0
        
        #Getting label
        label = self.classes[index]
        
        #Applying transforms on image
        if self.transforms:
            image = self.transforms(image)
        
        return image, label
     
    def __len__(self):
        return len(self.path_list)

In [149]:
train_dataset = Birds450Dataset(dataset_path, 'train', get_transform() )
valid_dataset = Birds450Dataset(dataset_path, 'valid', get_transform())
test_dataset = Birds450Dataset(dataset_path, 'test', get_transform())

In [150]:
BATCH_SIZE = 32

train_loader = DataLoader(train_dataset,batch_size=BATCH_SIZE,shuffle=True)
val_loader = DataLoader(valid_dataset,batch_size=BATCH_SIZE,shuffle=True)
test_loader = DataLoader(test_dataset,batch_size=BATCH_SIZE,shuffle=False)

##Preparing model

In [151]:
NUM_CLASSES = len(set(train_dataset.labels))
NUM_CLASSES

450

In [152]:
model = models.resnet50(pretrained=True)

# freezing all the parameters
for param in model.parameters():
    param.require_grad = False
    
model.fc = nn.Sequential(nn.Linear(model.fc.in_features,2048),
                         nn.ReLU(),
                         nn.Dropout(0.3),
                         nn.Linear(2048,1024),
                         nn.ReLU(),
                         nn.Dropout(0.3),
                         nn.Linear(1024,NUM_CLASSES))
model

/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and will be removed in 0.15, "
/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [153]:
device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)
print(device)

cuda:0


## Train process

In [154]:
def calculate_accuracy(target, prediction):
    preds = torch.max(prediction,dim=1)[1]
    matching = (preds == target).float()
    return matching.mean()

In [155]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(),lr=0.0001)


train_loss = []
train_accuracy = []

val_loss = []
val_accuracy = []

epochs = 10
model_path = "/content/gdrive/My Drive/birdResNet.pt"

In [156]:
### Training Code

for epoch in range(epochs):
    
    start = time.time()
    
    #Epoch Loss & Accuracy
    train_epoch_loss = []
    train_epoch_accuracy = []
    
    _iter = 1
    
    #Val Loss & Accuracy
    val_epoch_loss = []
    val_epoch_accuracy = []
    
    # Training
    print("Training...")
    for images, labels in train_loader:
        
        images = images.to(device)
        labels = labels.to(device)
        
        optimizer.zero_grad()
        
        preds = model(images)
        
        acc = calculate_accuracy(labels.cpu(), preds.cpu())
        
        loss = criterion(preds, labels)
        loss.backward()
        optimizer.step()
        
        loss_value = loss.item()
        train_epoch_loss.append(loss_value)
        train_epoch_accuracy.append(acc)
        
        if _iter % 500 == 0:
            print("> Iteration {} < ".format(_iter))
            print("Iter Loss = {}".format(round(loss_value, 4)))
            print("Iter Accuracy = {} % \n".format(acc))
        
        _iter += 1

    print("Validation...")
    for images, labels in val_loader:
        
        images = images.to(device)
        labels = labels.to(device)
        
        preds = model(images)
        
        acc = calculate_accuracy(labels.cpu(), preds.cpu())
        
        loss = criterion(preds, labels)
        
        loss_value = loss.item()
        val_epoch_loss.append(loss_value)
        val_epoch_accuracy.append(acc)
    
    
    train_epoch_loss = np.mean(train_epoch_loss)
    train_epoch_accuracy = np.mean(train_epoch_accuracy)
    
    val_epoch_loss = np.mean(val_epoch_loss)
    val_epoch_accuracy = np.mean(val_epoch_accuracy)
    
    end = time.time()
    
    train_loss.append(train_epoch_loss)
    train_accuracy.append(train_epoch_accuracy)
    
    val_loss.append(val_epoch_loss)
    val_accuracy.append(val_epoch_accuracy)
    
    #Print Epoch Statistics
    print("** Epoch {} ** - Epoch Time {}".format(epoch, int(end-start)))
    print("Train Loss = {}".format(round(train_epoch_loss, 4)))
    print("Train Accuracy = {} % \n".format(train_epoch_accuracy))
    print("Val Loss = {}".format(round(val_epoch_loss, 4)))
    print("Val Accuracy = {} % \n".format(val_epoch_accuracy))
    
torch.save(model.state_dict(), model_path)    

Training...
> Iteration 500 < 
Iter Loss = 3.2629
Iter Accuracy = 0.28125 % 

> Iteration 1000 < 
Iter Loss = 2.4842
Iter Accuracy = 0.46875 % 

> Iteration 1500 < 
Iter Loss = 1.3281
Iter Accuracy = 0.625 % 

> Iteration 2000 < 
Iter Loss = 0.9741
Iter Accuracy = 0.6875 % 

Validation...
** Epoch 0 ** - Epoch Time 831
Train Loss = 2.4194
Train Accuracy = 0.4430621564388275 % 

Val Loss = 0.8122
Val Accuracy = 0.7674295902252197 % 

Training...
> Iteration 500 < 
Iter Loss = 0.4245
Iter Accuracy = 0.84375 % 

> Iteration 1000 < 
Iter Loss = 1.0259
Iter Accuracy = 0.71875 % 

> Iteration 1500 < 
Iter Loss = 0.4862
Iter Accuracy = 0.875 % 

> Iteration 2000 < 
Iter Loss = 0.5199
Iter Accuracy = 0.78125 % 

Validation...
** Epoch 1 ** - Epoch Time 818
Train Loss = 0.674
Train Accuracy = 0.809697687625885 % 

Val Loss = 0.5009
Val Accuracy = 0.858186662197113 % 

Training...
> Iteration 500 < 
Iter Loss = 0.2687
Iter Accuracy = 0.90625 % 

> Iteration 1000 < 
Iter Loss = 0.4009
Iter Accura

In [158]:
test_loss = []
test_accuracy = []

model.eval()
for images, labels in test_loader:
        
  images = images.to(device)
  labels = labels.to(device)

  with torch.no_grad():      
      preds = model(images)
      
      acc = calculate_accuracy(labels.cpu(), preds.cpu())
      
      loss = criterion(preds, labels)
      
      loss_value = loss.item()

  test_loss.append(loss_value)
  test_accuracy.append(acc)

test_loss_val = np.mean(test_loss)
test_accuracy_val = np.mean(test_accuracy)

print("Test Loss = {}".format(round(test_loss_val, 4)))
print("Test Accuracy = {} % \n".format(test_accuracy_val))

Test Loss = 0.1379
Test Accuracy = 0.9656690359115601 % 



In [179]:
class_to_label_dict = class_dict[['class id', 'labels']].drop_duplicates(['class id'], keep = 'last')
class_to_label = dict()
for i in range(len(class_to_label_dict)):
  cl = class_to_label_dict['class id'].iloc[i]
  lab = class_to_label_dict['labels'].iloc[i]
  class_to_label[str(cl)] = lab

In [212]:
with open('/content/gdrive/My Drive/class_to_label.json', 'w') as fp:
  json.dump(class_to_label, fp)

In [191]:
img_tes =  cv2.imread('/content/train/COCKATOO/001.jpg', cv2.IMREAD_COLOR)
image = cv2.resize(img_tes,(224,224), interpolation=cv2.INTER_CUBIC)
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB).astype(np.float32)
image /= 255.0
image = image
convert_tensor = transforms.ToTensor()
image = convert_tensor(image)
image = torch.unsqueeze(image, 0)
with torch.no_grad():
  out = model(image.cuda())


In [198]:
class_id = int(torch.max(out,dim=1)[1][0])

In [199]:
class_to_label[str(class_id)]

'COCKATOO'

In [207]:
model2 = models.resnet50(pretrained=False)
model2.fc = nn.Sequential(nn.Linear(model2.fc.in_features,2048),
                         nn.ReLU(),
                         nn.Dropout(0.3),
                         nn.Linear(2048,1024),
                         nn.ReLU(),
                         nn.Dropout(0.3),
                         nn.Linear(1024,450))

model2.load_state_dict(torch.load(model_path,map_location=torch.device('cpu')))
model2.eval()
img_tes =  cv2.imread('/content/train/COCKATOO/001.jpg', cv2.IMREAD_COLOR)
image = cv2.resize(img_tes,(224,224), interpolation=cv2.INTER_CUBIC)
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB).astype(np.float32)
image /= 255.0
image = image
convert_tensor = transforms.ToTensor()
image = convert_tensor(image)
image = torch.unsqueeze(image, 0)
with torch.no_grad():
  out = model2(image)

class_id = int(torch.max(out,dim=1)[1][0])
class_to_label[str(class_id)]

'COCKATOO'

In [159]:
class_dict.head()

,class id,filepaths,labels,scientific label,data set
0,0,train/ABBOTTS BABBLER/001.jpg,ABBOTTS BABBLER,Malacocincla abbotti,train
1,0,train/ABBOTTS BABBLER/002.jpg,ABBOTTS BABBLER,Malacocincla abbotti,train
2,0,train/ABBOTTS BABBLER/003.jpg,ABBOTTS BABBLER,Malacocincla abbotti,train
3,0,train/ABBOTTS BABBLER/004.jpg,ABBOTTS BABBLER,Malacocincla abbotti,train
4,0,train/ABBOTTS BABBLER/005.jpg,ABBOTTS BABBLER,Malacocincla abbotti,train
